In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
!python -V

Python 3.9.7


In [10]:
import pickle
import pandas as pd
import numpy as np

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/anaconda3/envs/mlflow-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/envs/mlflow-env/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [15]:
year = 2023
month = 3
taxi_type = 'yellow'

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [12]:
print('standard deviation of the predicted duration for this dataset:', np.std(y_pred))

standard deviation of the predicted duration for this dataset: 6.247488852238703


In [16]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [25]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
# df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
# df_result['PULocationID'] = df['PULocationID']
# df_result['DOLocationID'] = df['DOLocationID']
# df_result['actual_duration'] = df['duration']
df_result['predicted_duration'] = y_pred
# df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
# df_result['model_version'] = df['ride_id']

output_file = f'output_{taxi_type}_{year:04d}-{month:02d}.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)